In [1]:
import re
import pandas as pd

# Load the CSV file into a DataFrame
short = pd.read_csv('../low-charge-density22.csv')
short['Class'] = 1

pos = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]
pos


,Sequence,Class
0,WWHSWWSTW,1
1,WWSYWWTQW,1
2,FHFHLHF,1
3,FCTMIPIPRCY,1
4,RVCFAIPLPICH,1
...,...,...
1761,QSRVIQGLVAGETAQQICED,1
1762,TCGETCFGGTCNTPGCTCDPWPICTDRGLP,1
1763,VCGETCEGGTCNTPGCSCSWPVCTRNGLP,1
1764,VCGETCVGGTCNTPGCSCSRPVCTRNGLP,1


In [2]:
import re
import pandas as pd
#from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
short = pd.read_csv('../neg-low-charge-density22.csv')
short['Class'] = 0
neg = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]
neg


neg = neg.sample(n=min(1766, len(neg)), random_state=40)
neg

,Sequence,Class
77822,ISLVDVGAEPAARENSSRRLLEFCSPEL,0
73513,ESEGDTDELAKLVGMGHFDPWVGDNV,0
66127,TSASSPSSKAPGSPS,0
27993,VVILGGVDDFVVFLVLVILGGVDDVIVLVE,0
20837,YKLDLQNAMLEYNKDLLHSKNG,0
...,...,...
20854,KGLSLFGVGNIHVDFKEQSLSC,0
37729,REAKMLETAGVTLIVARRQSSDREESSLQA,0
98646,QYNVLPQGWKGSPAI,0
79222,DWFSRQQLVMLVLFINISLALMFFKLLT,0


In [3]:
concatenated_df = pd.concat([pos,neg], ignore_index=True)
concatenated_df = concatenated_df.drop_duplicates(subset='Sequence', keep=False)
concatenated_df

,Sequence,Class
0,WWHSWWSTW,1
1,WWSYWWTQW,1
2,FHFHLHF,1
3,FCTMIPIPRCY,1
4,RVCFAIPLPICH,1
...,...,...
3527,KGLSLFGVGNIHVDFKEQSLSC,0
3528,REAKMLETAGVTLIVARRQSSDREESSLQA,0
3529,QYNVLPQGWKGSPAI,0
3530,DWFSRQQLVMLVLFINISLALMFFKLLT,0


In [4]:
import re
# Clean comma-separated sequences (take the first part)
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.split(',').str[0]

# Remove spaces
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.replace(' ', '', regex=False)

# Remove invalid amino acids
concatenated_df['Sequence'] = concatenated_df['Sequence'].apply(lambda x: re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', x))

#from sklearn.model_selection import train_test_split

# Remove sequences with issues
concatenated_df =concatenated_df[~concatenated_df['Sequence'].str.contains('[, ZBOUXJ ]')]

concatenated_df

,Sequence,Class
0,WWHSWWSTW,1
1,WWSYWWTQW,1
2,FHFHLHF,1
3,FCTMIPIPRCY,1
4,RVCFAIPLPICH,1
...,...,...
3527,KGLSLFGVGNIHVDFKEQSLSC,0
3528,REAKMLETAGVTLIVARRQSSDREESSLQA,0
3529,QYNVLPQGWKGSPAI,0
3530,DWFSRQQLVMLVLFINISLALMFFKLLT,0


In [5]:
concatenated_df = concatenated_df.reset_index(drop=True)
concatenated_df

,Sequence,Class
0,WWHSWWSTW,1
1,WWSYWWTQW,1
2,FHFHLHF,1
3,FCTMIPIPRCY,1
4,RVCFAIPLPICH,1
...,...,...
3525,KGLSLFGVGNIHVDFKEQSLSC,0
3526,REAKMLETAGVTLIVARRQSSDREESSLQA,0
3527,QYNVLPQGWKGSPAI,0
3528,DWFSRQQLVMLVLFINISLALMFFKLLT,0


In [7]:
import requests
import os
import time
import pandas as pd

# Configuration
OUTPUT_DIR = "ld//"
SLEEP_TIME = 5  # Seconds between requests
MAX_RETRIES = 3  # Retry attempts for failed requests
START_INDEX = 0  # <-- Change here to resume

# Create the output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_pdb_from_sequence(seq, out_file_path):
    """Get PDB structure from ESM Atlas API for a given amino acid sequence"""
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    headers = {
        "Content-Type": "text/plain"
    }

    # Sanitize sequence
    clean_seq = ''.join(filter(str.isalpha, seq)).upper()

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(url, data=clean_seq, headers=headers, timeout=30)
            if response.ok:
                with open(out_file_path, "w") as f:
                    f.write(response.text)
                return True
            else:
                print(f"Attempt {attempt + 1} failed for {clean_seq[:10]}... "
                      f"Status: {response.status_code}, Response: {response.text[:100]}")
                if response.status_code == 429:
                    time.sleep(SLEEP_TIME * (attempt + 2))  # Exponential backoff
        except Exception as e:
            print(f"Attempt {attempt + 1} error: {str(e)}")
            time.sleep(SLEEP_TIME)

    return False

# Initialize PDB_File column if missing
if 'PDB_File' not in concatenated_df.columns:
    concatenated_df['PDB_File'] = [None] * len(concatenated_df)

# Process each sequence from START_INDEX onward
for idx in range(START_INDEX, len(concatenated_df)):
    seq = concatenated_df.at[idx, 'Sequence']
    filename = f"peptide_{idx}.pdb"
    file_path = os.path.join(OUTPUT_DIR, filename)

    print(f"Processing sequence {idx + 1}/{len(concatenated_df)}: {seq[:15]}...")

    # Skip if already exists (optional)
    if os.path.exists(file_path):
        print(f"Already exists: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
        continue

    success = get_pdb_from_sequence(seq, file_path)
    if success:
        print(f"Successfully saved: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
    else:
        print(f"Failed to get PDB for index {idx}")
        concatenated_df.at[idx, 'PDB_File'] = None

    time.sleep(SLEEP_TIME)



Processing sequence 1/3530: WWHSWWSTW...
Already exists: ld//peptide_0.pdb
Processing sequence 2/3530: WWSYWWTQW...
Already exists: ld//peptide_1.pdb
Processing sequence 3/3530: FHFHLHF...
Already exists: ld//peptide_2.pdb
Processing sequence 4/3530: FCTMIPIPRCY...
Already exists: ld//peptide_3.pdb
Processing sequence 5/3530: RVCFAIPLPICH...
Already exists: ld//peptide_4.pdb
Processing sequence 6/3530: RVCYAIPLPICY...
Already exists: ld//peptide_5.pdb
Processing sequence 7/3530: RVCYAIPLPIC...
Already exists: ld//peptide_6.pdb
Processing sequence 8/3530: FLPLIGKVLSGIL...
Already exists: ld//peptide_7.pdb
Processing sequence 9/3530: FLPLLGRVLSGLL...
Already exists: ld//peptide_8.pdb
Processing sequence 10/3530: GLFDIIKKIAESI...
Already exists: ld//peptide_9.pdb
Processing sequence 11/3530: GLFDIIKKIAESF...
Already exists: ld//peptide_10.pdb
Processing sequence 12/3530: FISAIASMLGKFL...
Already exists: ld//peptide_11.pdb
Processing sequence 13/3530: FLSAIASMLGKFL...
Already exists: ld//p

Already exists: ld//peptide_1065.pdb
Processing sequence 1067/3530: IHHIHHHIHHIHHHI...
Already exists: ld//peptide_1066.pdb
Processing sequence 1068/3530: IHHIHHIIHHIHHII...
Already exists: ld//peptide_1067.pdb
Processing sequence 1069/3530: IHHIHHIIHHIHHII...
Already exists: ld//peptide_1068.pdb
Processing sequence 1070/3530: GGHGLFDVIKKVASV...
Already exists: ld//peptide_1069.pdb
Processing sequence 1071/3530: GLWSKIKDAAKTAGK...
Already exists: ld//peptide_1070.pdb
Processing sequence 1072/3530: SLWSSIKDMAAAAGR...
Already exists: ld//peptide_1071.pdb
Processing sequence 1073/3530: SLWSSIKDMAAAAGR...
Already exists: ld//peptide_1072.pdb
Processing sequence 1074/3530: SLRSSIKDMAAAAGR...
Already exists: ld//peptide_1073.pdb
Processing sequence 1075/3530: VMWCYVFGYGFNCAV...
Already exists: ld//peptide_1074.pdb
Processing sequence 1076/3530: GIKEMLCNMACAQTV...
Already exists: ld//peptide_1075.pdb
Processing sequence 1077/3530: DLTTKLWSSWGYYLG...
Already exists: ld//peptide_1076.pdb
Proces

Already exists: ld//peptide_2162.pdb
Processing sequence 2164/3530: IGAIFSATDSVCTLQ...
Already exists: ld//peptide_2163.pdb
Processing sequence 2165/3530: LFSSYIFN...
Already exists: ld//peptide_2164.pdb
Processing sequence 2166/3530: KRALDAAYCFRNGQD...
Already exists: ld//peptide_2165.pdb
Processing sequence 2167/3530: SGGYGYLLEPLWWVG...
Already exists: ld//peptide_2166.pdb
Processing sequence 2168/3530: NTRELRIAVLLV...
Already exists: ld//peptide_2167.pdb
Processing sequence 2169/3530: RGMMYYRRALKLQAF...
Already exists: ld//peptide_2168.pdb
Processing sequence 2170/3530: ANKGLGLTAEIVDHC...
Already exists: ld//peptide_2169.pdb
Processing sequence 2171/3530: EMQYVSFKAKGGTFY...
Already exists: ld//peptide_2170.pdb
Processing sequence 2172/3530: DGLGFKPDSDGRDSK...
Already exists: ld//peptide_2171.pdb
Processing sequence 2173/3530: HAKCTSQGIHGSCCL...
Already exists: ld//peptide_2172.pdb
Processing sequence 2174/3530: VIFFVSLFIFGFLSN...
Already exists: ld//peptide_2173.pdb
Processing seque

Already exists: ld//peptide_3181.pdb
Processing sequence 3183/3530: IIHSFGHYPSSIKFG...
Already exists: ld//peptide_3182.pdb
Processing sequence 3184/3530: PLPQTRGNSAGSEES...
Already exists: ld//peptide_3183.pdb
Processing sequence 3185/3530: VPTALGTALDINLSN...
Already exists: ld//peptide_3184.pdb
Processing sequence 3186/3530: VGDASGMQSPVSFGG...
Already exists: ld//peptide_3185.pdb
Processing sequence 3187/3530: QEQKDKENPPPSISL...
Already exists: ld//peptide_3186.pdb
Processing sequence 3188/3530: SLVNFYGNIGKTLLS...
Already exists: ld//peptide_3187.pdb
Processing sequence 3189/3530: ILIVLIEEVLIVDVE...
Already exists: ld//peptide_3188.pdb
Processing sequence 3190/3530: LVFFLEGNQWNERGR...
Already exists: ld//peptide_3189.pdb
Processing sequence 3191/3530: VPEDWVCPLCGVGKD...
Already exists: ld//peptide_3190.pdb
Processing sequence 3192/3530: HFGVSRPTLDKALKT...
Already exists: ld//peptide_3191.pdb
Processing sequence 3193/3530: CGSGCGVVMIAVIVV...
Already exists: ld//peptide_3192.pdb
Proces

In [6]:
import os

OUTPUT_DIR = "../ld/"
existing_files = os.listdir(OUTPUT_DIR)

for file in existing_files:
    if file.endswith(".pdb") and file.startswith("peptide_"):
        try:
            idx = int(file.replace("peptide_", "").replace(".pdb", ""))
            file_path = os.path.join(OUTPUT_DIR, file)
            if 0 <= idx < len(concatenated_df):
                concatenated_df.at[idx, 'PDB_File'] = file_path
        except ValueError:
            continue  # In case there's a filename that doesn't match the pattern


concatenated_df.to_csv("peptides_with_pdb_paths_fixed.csv", index=False)
print("✅ PDB_File column updated from existing files.")
concatenated_df

✅ PDB_File column updated from existing files.


,Sequence,Class,PDB_File
0,WWHSWWSTW,1,../ld/peptide_0.pdb
1,WWSYWWTQW,1,../ld/peptide_1.pdb
2,FHFHLHF,1,../ld/peptide_2.pdb
3,FCTMIPIPRCY,1,../ld/peptide_3.pdb
4,RVCFAIPLPICH,1,../ld/peptide_4.pdb
...,...,...,...
3525,KGLSLFGVGNIHVDFKEQSLSC,0,../ld/peptide_3525.pdb
3526,REAKMLETAGVTLIVARRQSSDREESSLQA,0,../ld/peptide_3526.pdb
3527,QYNVLPQGWKGSPAI,0,../ld/peptide_3527.pdb
3528,DWFSRQQLVMLVLFINISLALMFFKLLT,0,../ld/peptide_3528.pdb


In [7]:
from Bio.PDB import PDBParser, DSSP
import pandas as pd
import numpy as np
import os

# Path to folder containing PDB files
pdb_directory = "../ld/./"  

# Initialize DSSP feature columns
dssp_columns = ['Mean_RSA', 'Mean_Phi', 'Mean_Psi']
for col in dssp_columns:
    concatenated_df[col] = np.nan

parser = PDBParser(QUIET=True)

# Loop to calculate RSA, Phi, Psi features
for idx in concatenated_df.index:
    pdb_filename = f"peptide_{idx}.pdb"
    pdb_path = os.path.join(pdb_directory, pdb_filename)

    if not os.path.exists(pdb_path):
        print(f"File not found: {pdb_path}")
        continue

    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_path)
        model = structure[0]
        dssp = DSSP(model, pdb_path)

        rsa_list = []
        phi_list = []
        psi_list = []

        for res in dssp:
            rsa = res[3]
            phi = res[4]
            psi = res[5]

            rsa_list.append(rsa)
            phi_list.append(phi)
            psi_list.append(psi)

        # Compute means
        concatenated_df.at[idx, 'Mean_RSA'] = np.mean(rsa_list) if rsa_list else np.nan
        concatenated_df.at[idx, 'Mean_Phi'] = np.mean(phi_list) if phi_list else np.nan
        concatenated_df.at[idx, 'Mean_Psi'] = np.mean(psi_list) if psi_list else np.nan

    except Exception as e:
        print(f"Error processing index {idx} - file: {pdb_path}: {e}")


In [8]:
from Bio.PDB import PDBParser, DSSP
import os

# Columns to store DSSP secondary structure percentages
helices = []
sheets = []
coils = []

parser = PDBParser()

for idx in concatenated_df.index:
    pdb_file = f"../ld/peptide_{idx}.pdb"
    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_file)
        model = structure[0]
        dssp = DSSP(model, pdb_file)
        
        ss_list = [dssp[key][2] for key in dssp.keys()]
        total = len(ss_list)
        
        helix_count = sum(s in "HGI" for s in ss_list)
        sheet_count = sum(s in "BE" for s in ss_list)
        coil_count  = sum(s in "ST " for s in ss_list)  # includes turn, bend, and loops
        
        helices.append(helix_count / total * 100)
        sheets.append(sheet_count / total * 100)
        coils.append(coil_count / total * 100)
    except Exception as e:
        print(f"Failed to process {pdb_file}: {e}")
        helices.append(None)
        sheets.append(None)
        coils.append(None)

# Add DSSP % columns to DataFrame
concatenated_df['%Helix'] = helices
concatenated_df['%Sheet'] = sheets
concatenated_df['%Coil'] = coils


In [9]:
import mdtraj as md
import pandas as pd
import numpy as np
import os

# Folder containing your PDBs
pdb_folder = '../ld/'

# Initialize columns
concatenated_df['RoG'] = np.nan
concatenated_df['SASA'] = np.nan
concatenated_df['Compactness'] = np.nan

# Loop through each peptide
for idx in concatenated_df.index:
    pdb_path = os.path.join(pdb_folder, f"peptide_{idx}.pdb")

    if not os.path.exists(pdb_path):
        print(f"Missing: {pdb_path}")
        continue

    try:
        traj = md.load(pdb_path)
        
        # Radius of Gyration (RoG)
        rog = md.compute_rg(traj)[0]
        concatenated_df.at[idx, 'RoG'] = rog

        # Solvent Accessible Surface Area (SASA)
        sasa = md.shrake_rupley(traj)[0].sum()
        concatenated_df.at[idx, 'SASA'] = sasa

        # Compactness = Mass / SASA (or optionally 1/SASA per atom)
        n_atoms = traj.n_atoms
        compactness = n_atoms / sasa if sasa > 0 else np.nan
        concatenated_df.at[idx, 'Compactness'] = compactness

    except Exception as e:
        print(f"Error at index {idx}: {e}")


/home/hrushikesh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
cleaned_df = concatenated_df
cleaned_df

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness
0,WWHSWWSTW,1,../ld/peptide_0.pdb,0.881183,-37.833333,65.700000,0.000000,0.000000,33.333333,0.794872,16.208475,6.107916
1,WWSYWWTQW,1,../ld/peptide_1.pdb,0.798940,-34.588889,18.066667,55.555556,0.000000,22.222222,0.636526,14.778220,7.037383
2,FHFHLHF,1,../ld/peptide_2.pdb,0.890459,-18.028571,132.371429,0.000000,0.000000,0.000000,0.717753,12.544179,5.659996
3,FCTMIPIPRCY,1,../ld/peptide_3.pdb,0.763699,-32.036364,111.227273,0.000000,36.363636,27.272727,0.665534,14.598289,6.233607
4,RVCFAIPLPICH,1,../ld/peptide_4.pdb,0.787609,-28.325000,97.308333,0.000000,0.000000,16.666667,0.723347,15.690041,5.991062
...,...,...,...,...,...,...,...,...,...,...,...,...
3525,KGLSLFGVGNIHVDFKEQSLSC,0,../ld/peptide_3525.pdb,0.886621,-21.031818,109.736364,0.000000,0.000000,18.181818,1.301907,31.030502,5.349575
3526,REAKMLETAGVTLIVARRQSSDREESSLQA,0,../ld/peptide_3526.pdb,0.843640,-41.630000,71.780000,13.333333,0.000000,13.333333,1.656337,41.729187,5.535694
3527,QYNVLPQGWKGSPAI,0,../ld/peptide_3527.pdb,0.821952,14.686667,92.553333,0.000000,0.000000,26.666667,0.937227,19.915075,5.874946
3528,DWFSRQQLVMLVLFINISLALMFFKLLT,0,../ld/peptide_3528.pdb,0.664229,-51.446429,-25.850000,92.857143,0.000000,0.000000,1.308744,32.145687,7.403793


In [11]:
import pandas as pd
import peptides


# Define the peptide descriptors you want to calculate
descriptors = [
    'frequencies',
    'aliphatic_index',
    'boman',
    'charge',
    'isoelectric_point',
    'hydrophobic_moment',
    'hydrophobicity',
    'instability_index',
    'mass_shift',
    'molecular_weight',
    'mz',
    'structural_class'
]

# Create a dictionary to store descriptor values
descriptor_values_dict = {}

# Iterate through each descriptor and calculate values for each sequence
for descriptor in descriptors:
    descriptor_values = []
    for sequence in cleaned_df['Sequence']:
        peptide = peptides.Peptide(sequence)
        value = getattr(peptide, descriptor)()
        descriptor_values.append(value)
    descriptor_values_dict[descriptor] = descriptor_values

# Create a new DataFrame using the descriptor values dictionary
descriptors = pd.DataFrame(descriptor_values_dict)

In [12]:
import pandas as pd


# Expand 'frequencies' column into separate columns
amino_acid_frequencies = pd.DataFrame(descriptors['frequencies'].tolist()).fillna(0)

# Combine the expanded DataFrame with the original DataFrame
descriptors_df = pd.concat([descriptors, amino_acid_frequencies], axis=1)

# Drop the original 'frequencies' column
descriptors_df.drop(columns=['frequencies'], inplace=True)



In [72]:
descriptors_df

,aliphatic_index,boman,charge,isoelectric_point,hydrophobic_moment,hydrophobicity,instability_index,mass_shift,molecular_weight,mz,...,T,W,Y,V,O,U,B,Z,J,X
0,0.000000,2.644444e-01,0.088893,7.550324,0.367203,-1.111111,18.500000,0.000000,1361.48384,681.296156,...,0.111111,0.555556,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,-9.868649e-17,-0.002866,6.093198,0.314121,-1.200000,-27.711111,0.000000,1428.57124,714.811641,...,0.111111,0.555556,0.111111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,55.714286,1.714286e-02,0.270712,7.811003,0.165449,0.371429,-21.057143,0.000000,984.12774,492.745566,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,70.909091,-9.090909e-03,0.873181,8.222829,0.317083,0.818182,26.636364,6.020129,1343.68284,729.340540,...,0.090909,0.000000,0.090909,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,130.000000,-5.475000e-01,0.964941,8.242952,0.309160,1.308333,49.983333,6.020129,1368.72104,741.889035,...,0.000000,0.000000,0.000000,0.083333,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,97.272727,7.204545e-01,0.028550,7.359368,0.329181,0.172727,50.004545,12.040258,2378.72824,1218.122993,...,0.000000,0.000000,0.000000,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
3526,84.666667,3.090667e+00,0.005223,6.697892,0.386656,-0.600000,93.513333,30.100645,3332.73684,1666.372951,...,0.066667,0.000000,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0
3527,78.000000,4.606667e-01,0.996839,9.298301,0.412307,-0.473333,74.686667,6.020129,1657.88934,829.438456,...,0.000000,0.066667,0.066667,0.066667,0.0,0.0,0.0,0.0,0.0,0.0
3528,149.642857,-6.014286e-01,0.998132,9.695025,0.371132,1.257143,37.571429,12.040258,3388.17554,1693.942176,...,0.035714,0.035714,0.000000,0.071429,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem


# Create a new DataFrame to store the results
d4 = pd.DataFrame()

def calculate_tpsa(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate TPSA using AllChem method
        tpsa = AllChem.CalcTPSA(peptide_molecule)

        return tpsa
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create a new 'TPSA' column
#new_pos['Sequence'] = cleaned_df['Sequence']
d4['TPSA'] = cleaned_df['Sequence'].apply(calculate_tpsa)

# Display the new DataFrame with TPSA values
d4

,TPSA
0,464.44
1,478.85
2,323.96
3,439.10
4,456.42
...,...
3525,976.61
3526,1574.90
3527,664.68
3528,1179.99


In [14]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Crippen, Lipinski, Descriptors


# Create a new DataFrame to store the results
d6 = pd.DataFrame()

def calculate_descriptors(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate various molecular descriptors
        heavy_atom_count = Descriptors.HeavyAtomCount(peptide_molecule)
        logp = Crippen.MolLogP(peptide_molecule)
        mol_logp = Crippen.MolLogP(peptide_molecule)
    
        fraction_csp3 = Lipinski.FractionCSP3(peptide_molecule)
        bertz_ct = Descriptors.BertzCT(peptide_molecule)

        return heavy_atom_count, logp, mol_logp, fraction_csp3, bertz_ct
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d6[['HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT']] = cleaned_df['Sequence'].apply(calculate_descriptors).apply(pd.Series)

# Display the new DataFrame with descriptor values
d6

,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT
0,100.0,1.22790,1.22790,0.267606,4760.949999
1,105.0,2.48360,2.48360,0.263158,4996.440381
2,72.0,0.97870,0.97870,0.333333,2651.647234
3,92.0,-1.20133,-1.20133,0.606557,2846.576973
4,95.0,-1.11093,-1.11093,0.650794,2946.026419
...,...,...,...,...,...
3525,167.0,-9.93400,-9.93400,0.613208,5621.264957
3526,232.0,-20.20422,-20.20422,0.700730,7694.198003
3527,118.0,-5.57120,-5.57120,0.584416,4064.429843
3528,239.0,-1.92553,-1.92553,0.603659,8803.106871


In [15]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors


# Create a new DataFrame to store the results
d7 = pd.DataFrame()

def calculate_polarity(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate descriptors related to polarity
        num_h_acceptors = Lipinski.NumHAcceptors(peptide_molecule)
        num_h_donors = Lipinski.NumHDonors(peptide_molecule)
        

        return num_h_acceptors, num_h_donors
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for polarity descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d7[['NumHAcceptors', 'NumHDonors',]] = cleaned_df['Sequence'].apply(calculate_polarity).apply(pd.Series)

# Display the new DataFrame with polarity descriptors
d7

,NumHAcceptors,NumHDonors
0,14,19
1,14,19
2,11,11
3,18,17
4,17,17
...,...,...
3525,34,34
3526,50,57
3527,22,21
3528,40,41


In [16]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors


# Create a new DataFrame to store the results
df8 = pd.DataFrame()

def calculate_amino_acid_properties(peptide_sequence):
    ng_count = peptide_sequence.count('C(N)N')
    npa_count = peptide_sequence.count('C(N)C')
    nncaa_count = peptide_sequence.count('D') + peptide_sequence.count('E')

    return ng_count, npa_count, nncaa_count
# Apply the function to the 'Sequence' column and create new columns for amino acid properties
df8[['NG', 'NPA', 'NNCAA']] = cleaned_df['Sequence'].apply(calculate_amino_acid_properties).apply(pd.Series)

# Display the DataFrame with amino acid properties
df8

,NG,NPA,NNCAA
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
3525,0,0,2
3526,0,0,5
3527,0,0,0
3528,0,0,1


In [17]:
cleaned_df.reset_index(drop=True, inplace=True)
descriptors_df.reset_index(drop=True, inplace=True)
d4.reset_index(drop=True, inplace=True)
d6.reset_index(drop=True, inplace=True)
d7.reset_index(drop=True, inplace=True)
df8.reset_index(drop=True, inplace=True)


In [18]:
ll = pd.concat([cleaned_df,descriptors_df,d4,d6,d7,df8], axis=1)
ll = ll.fillna(0)
ll

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,WWHSWWSTW,1,../ld/peptide_0.pdb,0.881183,-37.833333,65.700000,0.000000,0.000000,33.333333,0.794872,...,100.0,1.22790,1.22790,0.267606,4760.949999,14,19,0,0,0
1,WWSYWWTQW,1,../ld/peptide_1.pdb,0.798940,-34.588889,18.066667,55.555556,0.000000,22.222222,0.636526,...,105.0,2.48360,2.48360,0.263158,4996.440381,14,19,0,0,0
2,FHFHLHF,1,../ld/peptide_2.pdb,0.890459,-18.028571,132.371429,0.000000,0.000000,0.000000,0.717753,...,72.0,0.97870,0.97870,0.333333,2651.647234,11,11,0,0,0
3,FCTMIPIPRCY,1,../ld/peptide_3.pdb,0.763699,-32.036364,111.227273,0.000000,36.363636,27.272727,0.665534,...,92.0,-1.20133,-1.20133,0.606557,2846.576973,18,17,0,0,0
4,RVCFAIPLPICH,1,../ld/peptide_4.pdb,0.787609,-28.325000,97.308333,0.000000,0.000000,16.666667,0.723347,...,95.0,-1.11093,-1.11093,0.650794,2946.026419,17,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,KGLSLFGVGNIHVDFKEQSLSC,0,../ld/peptide_3525.pdb,0.886621,-21.031818,109.736364,0.000000,0.000000,18.181818,1.301907,...,167.0,-9.93400,-9.93400,0.613208,5621.264957,34,34,0,0,2
3526,REAKMLETAGVTLIVARRQSSDREESSLQA,0,../ld/peptide_3526.pdb,0.843640,-41.630000,71.780000,13.333333,0.000000,13.333333,1.656337,...,232.0,-20.20422,-20.20422,0.700730,7694.198003,50,57,0,0,5
3527,QYNVLPQGWKGSPAI,0,../ld/peptide_3527.pdb,0.821952,14.686667,92.553333,0.000000,0.000000,26.666667,0.937227,...,118.0,-5.57120,-5.57120,0.584416,4064.429843,22,21,0,0,0
3528,DWFSRQQLVMLVLFINISLALMFFKLLT,0,../ld/peptide_3528.pdb,0.664229,-51.446429,-25.850000,92.857143,0.000000,0.000000,1.308744,...,239.0,-1.92553,-1.92553,0.603659,8803.106871,40,41,0,0,1


In [20]:
from Bio import SeqIO

# Save sequences to a FASTA file for CD-HIT
with open("all_sequences.fasta", "w") as f:
    for i, seq in enumerate(ll["Sequence"]):
        f.write(f">seq_{i}\n{seq}\n")

In [21]:
import subprocess

identity = 0.95  # 90% identity
cmd = f"cd-hit -i all_sequences.fasta -o clustered.fasta -c {identity}"

subprocess.run(cmd, shell=True)

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i all_sequences.fasta -o clustered.fasta -c
         0.95

Started: Mon Dec  1 22:28:27 2025
                            Output                              
----------------------------------------------------------------
total seq: 3318
longest and shortest : 39 and 11
Total letters: 76427
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90441901

comparing sequences from          0  to       3318
...
     3318  finished       2945  clusters

Approximated maximum memory consumption: 76M
writing new database
writing clustering information
program completed !

Total CPU time 0.09


CompletedProcess(args='cd-hit -i all_sequences.fasta -o clustered.fasta -c 0.95', returncode=0)

In [22]:
representative_ids = []

with open("clustered.fasta") as f:
    for line in f:
        if line.startswith(">"):
            rep_id = line.strip().replace(">", "")
            representative_ids.append(rep_id)


In [23]:
kept_indices = [int(x.split("_")[1]) for x in representative_ids]

In [24]:
ll_filtered = ll.iloc[kept_indices].reset_index(drop=True)
ll_filtered

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,FCTMIPIPRCY,1,../ld/peptide_3.pdb,0.763699,-32.036364,111.227273,0.000000,36.363636,27.272727,0.665534,...,92.0,-1.20133,-1.20133,0.606557,2846.576973,18,17,0,0,0
1,RVCFAIPLPICH,1,../ld/peptide_4.pdb,0.787609,-28.325000,97.308333,0.000000,0.000000,16.666667,0.723347,...,95.0,-1.11093,-1.11093,0.650794,2946.026419,17,17,0,0,0
2,RVCYAIPLPICY,1,../ld/peptide_5.pdb,0.760222,-50.091667,54.633333,0.000000,0.000000,50.000000,0.727283,...,98.0,-0.42283,-0.42283,0.621212,3092.183298,18,18,0,0,0
3,FLPLIGKVLSGIL,1,../ld/peptide_7.pdb,0.833414,-50.653846,56.330769,38.461538,0.000000,15.384615,0.961075,...,97.0,0.28150,0.28150,0.720588,2752.085204,16,15,0,0,0
4,FLPLLGRVLSGLL,1,../ld/peptide_8.pdb,0.811572,-63.207692,23.261538,30.769231,0.000000,38.461538,0.831623,...,99.0,-0.58423,-0.58423,0.705882,2857.167679,16,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,KGLSLFGVGNIHVDFKEQSLSC,0,../ld/peptide_3525.pdb,0.886621,-21.031818,109.736364,0.000000,0.000000,18.181818,1.301907,...,167.0,-9.93400,-9.93400,0.613208,5621.264957,34,34,0,0,2
2941,REAKMLETAGVTLIVARRQSSDREESSLQA,0,../ld/peptide_3526.pdb,0.843640,-41.630000,71.780000,13.333333,0.000000,13.333333,1.656337,...,232.0,-20.20422,-20.20422,0.700730,7694.198003,50,57,0,0,5
2942,QYNVLPQGWKGSPAI,0,../ld/peptide_3527.pdb,0.821952,14.686667,92.553333,0.000000,0.000000,26.666667,0.937227,...,118.0,-5.57120,-5.57120,0.584416,4064.429843,22,21,0,0,0
2943,DWFSRQQLVMLVLFINISLALMFFKLLT,0,../ld/peptide_3528.pdb,0.664229,-51.446429,-25.850000,92.857143,0.000000,0.000000,1.308744,...,239.0,-1.92553,-1.92553,0.603659,8803.106871,40,41,0,0,1


In [25]:
categorical_df = ll_filtered.copy()

# Convert the "structural_class" column to categorical
categorical_df['structural_class'] = categorical_df['structural_class'].astype('category')

# Assign categorical codes to the categories
categorical_df['structural_class'] = categorical_df['structural_class'].cat.codes
categorical_df
categorical_df.to_csv('Low-charge-density.csv', index=False)

In [26]:
# Then drop unnecessary columns
X = categorical_df.drop(['Class','Sequence','PDB_File','B','Z','X','O','J','U'], axis=1)
X = X.fillna(0)
y = categorical_df['Class']
y=y.astype('int')

X.columns

Index(['Mean_RSA', 'Mean_Phi', 'Mean_Psi', '%Helix', '%Sheet', '%Coil', 'RoG',
       'SASA', 'Compactness', 'aliphatic_index', 'boman', 'charge',
       'isoelectric_point', 'hydrophobic_moment', 'hydrophobicity',
       'instability_index', 'mass_shift', 'molecular_weight', 'mz',
       'structural_class', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
       'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'TPSA',
       'HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT',
       'NumHAcceptors', 'NumHDonors', 'NG', 'NPA', 'NNCAA'],
      dtype='object')

In [27]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume X is your DataFrame
scaler = MinMaxScaler()

# Apply Min-Max Normalization to all columns in X
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display the normalized DataFrame
print("Normalized DataFrame (Min-Max Scaling):")
X_normalized


Normalized DataFrame (Min-Max Scaling):


,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.623700,0.404742,0.696032,0.000000,0.436364,0.371901,0.032792,0.049629,0.254557,0.270328,...,0.082734,0.805739,0.805739,0.569482,0.073933,0.089286,0.093333,0.0,0.0,0.000000
1,0.669652,0.431995,0.629357,0.000000,0.000000,0.227273,0.053943,0.073042,0.212909,0.495601,...,0.093525,0.807988,0.807988,0.666414,0.081505,0.071429,0.093333,0.0,0.0,0.000000
2,0.617017,0.272161,0.424935,0.000000,0.000000,0.681818,0.055383,0.075041,0.239472,0.495601,...,0.104317,0.825109,0.825109,0.601594,0.092633,0.089286,0.106667,0.0,0.0,0.000000
3,0.757684,0.268033,0.433066,0.405405,0.000000,0.209790,0.140917,0.112774,0.123836,0.771261,...,0.100719,0.842634,0.842634,0.819350,0.066738,0.053571,0.066667,0.0,0.0,0.000000
4,0.715706,0.175849,0.274657,0.324324,0.000000,0.524476,0.093556,0.099514,0.178460,0.771261,...,0.107914,0.821093,0.821093,0.787126,0.074739,0.053571,0.093333,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,0.859943,0.485550,0.688890,0.000000,0.000000,0.247934,0.265612,0.402020,0.102756,0.370834,...,0.352518,0.588456,0.588456,0.584054,0.285195,0.375000,0.320000,0.0,0.0,0.153846
2941,0.777338,0.334295,0.507071,0.140541,0.000000,0.181818,0.395282,0.631454,0.134716,0.322776,...,0.586331,0.332917,0.332917,0.775836,0.443026,0.660714,0.626667,0.0,0.0,0.384615
2942,0.735656,0.747834,0.606580,0.000000,0.000000,0.363636,0.132192,0.163648,0.192970,0.297361,...,0.176259,0.697010,0.697010,0.520964,0.166659,0.160714,0.146667,0.0,0.0,0.000000
2943,0.432529,0.262213,0.039403,0.978764,0.000000,0.000000,0.268113,0.425935,0.455495,0.570486,...,0.611511,0.787720,0.787720,0.563130,0.527457,0.482143,0.413333,0.0,0.0,0.076923


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [86]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
scaler = MinMaxScaler()

# Apply Min-Max Normalization to all columns in X
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display the normalized DataFrame
print("Normalized DataFrame (Min-Max Scaling):")
X_normalized

Normalized DataFrame (Min-Max Scaling):


,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.815747,0.331091,0.470894,0.000000,0.000000,0.454545,0.102436,0.142697,0.259947,0.000000,...,0.171141,0.822142,0.822142,0.075475,0.258213,0.098361,0.1750,0.0,0.0,0.000000
1,0.663965,0.352870,0.246087,0.585586,0.000000,0.303030,0.045911,0.113985,0.413937,0.000000,...,0.187919,0.851797,0.851797,0.068078,0.275257,0.098361,0.1750,0.0,0.0,0.000000
2,0.832866,0.464038,0.785551,0.000000,0.000000,0.000000,0.074907,0.069138,0.185737,0.171722,...,0.077181,0.816257,0.816257,0.184783,0.105541,0.049180,0.0750,0.0,0.0,0.000000
3,0.598925,0.370005,0.685761,0.000000,0.436364,0.371901,0.056266,0.110373,0.280771,0.218555,...,0.144295,0.764772,0.764772,0.639166,0.119650,0.163934,0.1500,0.0,0.0,0.000000
4,0.643052,0.394919,0.620070,0.000000,0.000000,0.227273,0.076904,0.132290,0.240587,0.400685,...,0.154362,0.766907,0.766907,0.712733,0.126848,0.147541,0.1500,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,0.825784,0.443877,0.678725,0.000000,0.000000,0.247934,0.283435,0.440240,0.134308,0.299813,...,0.395973,0.558537,0.558537,0.650226,0.320482,0.426230,0.3625,0.0,0.0,0.153846
3526,0.746460,0.305604,0.499589,0.140541,0.000000,0.181818,0.409958,0.655010,0.165143,0.260959,...,0.614094,0.315990,0.315990,0.795779,0.470522,0.688525,0.6500,0.0,0.0,0.384615
3527,0.706434,0.683651,0.597629,0.000000,0.000000,0.363636,0.153253,0.217105,0.221350,0.240411,...,0.231544,0.661571,0.661571,0.602343,0.207798,0.229508,0.2000,0.0,0.0,0.000000
3528,0.415348,0.239708,0.038821,0.978764,0.000000,0.000000,0.285876,0.462627,0.474643,0.461228,...,0.637584,0.747669,0.747669,0.634345,0.550785,0.524590,0.4500,0.0,0.0,0.076923


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)


In [31]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

# Create the XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=60, learning_rate=0.2, max_depth=2,random_state=42)

# Perform 10-fold cross-validation
cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=10)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Fit the model on the entire training set
xgb_classifier.fit(X_train, y_train)

# Predictions on the training set
y_train_pred = xgb_classifier.predict(X_train)

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

# Predictions on the test set
y_test_pred = xgb_classifier.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Calculate precision on the test set
precision = precision_score(y_test, y_test_pred)
print("Precision:", precision)

# Calculate sensitivity (recall) on the test set
recall = recall_score(y_test, y_test_pred)
print("Sensitivity (Recall):", recall)

# Calculate ROC AUC score on the test set
roc_auc = roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:,1])
print("ROC AUC:", roc_auc)





Cross-Validation Scores: [0.92372881 0.90677966 0.86016949 0.90254237 0.88983051 0.8559322
 0.88085106 0.91914894 0.8893617  0.9106383 ]
Mean CV Accuracy: 0.8938983050847458
Train Accuracy: 0.9227504244482173
Test Accuracy: 0.8964346349745331
Precision: 0.9094650205761317
Sensitivity (Recall): 0.85
ROC AUC: 0.9470657002571896
